In [9]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
#from plotly.offline import init_notebook_mode, iplot
#import plotly.graph_objs as go
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [11]:
df = pd.read_csv("files\\data_preprocessing\\korean_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'korean']

In [12]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)  
data

,emd_cd,korean
base_date,,
2018-01-31,50110250,7.209306e+06
2018-02-28,50110250,5.579125e+06
2018-03-31,50110250,7.615021e+06
2018-04-30,50110250,8.498349e+06
2018-05-31,50110250,8.916782e+06
...,...,...
2021-02-28,50130620,4.321265e+06
2021-03-31,50130620,4.744309e+06
2021-04-30,50130620,5.236706e+06


In [13]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50110256  ##각행정동코드

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,korean
base_date,
2018-01-31,5.853572e+06
2018-02-28,5.046954e+06
2018-03-31,6.188935e+06
2018-04-30,7.037538e+06
2018-05-31,7.325580e+06
2018-06-30,6.999620e+06
2018-07-31,7.801604e+06
2018-08-31,8.303571e+06
2018-09-30,6.749093e+06


In [14]:
# series 형으로 변환
sub_df.to_dict('series')

{'korean': base_date
 2018-01-31    5.853572e+06
 2018-02-28    5.046954e+06
 2018-03-31    6.188935e+06
 2018-04-30    7.037538e+06
 2018-05-31    7.325580e+06
 2018-06-30    6.999620e+06
 2018-07-31    7.801604e+06
 2018-08-31    8.303571e+06
 2018-09-30    6.749093e+06
 2018-10-31    6.622633e+06
 2018-11-30    5.535023e+06
 2018-12-31    5.424464e+06
 2019-01-31    5.845020e+06
 2019-02-28    5.697814e+06
 2019-03-31    6.241396e+06
 2019-04-30    6.733536e+06
 2019-05-31    7.143541e+06
 2019-06-30    6.931169e+06
 2019-07-31    7.635277e+06
 2019-08-31    8.430994e+06
 2019-09-30    6.236580e+06
 2019-10-31    6.687875e+06
 2019-11-30    5.820780e+06
 2019-12-31    5.925513e+06
 2020-01-31    6.287627e+06
 2020-02-29    4.380514e+06
 2020-03-31    4.855240e+06
 2020-04-30    4.761806e+06
 2020-05-31    5.808829e+06
 2020-06-30    6.037154e+06
 2020-07-31    7.119620e+06
 2020-08-31    8.258599e+06
 2020-09-30    5.827093e+06
 2020-10-31    6.971611e+06
 2020-11-30    6.808006e+06

# ARIMA

In [15]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)
#sub_df_float.tail()

In [16]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))
    
##### 여기서 p-value 확인 -> 0.05보다 크면 아래로 내려가서 차분하기 / 작으면 차분 생략(아래 셀 생략) #####

ADF Statistic : -3.814582
p-value : 0.002762
Critical Values :
	1%: -3.615509
	5%: -2.941262
	10%: -2.609200


In [17]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

##### 여기서 아래쪽에 'Best model:  ARIMA(0,1,0)(0,0,0)[1] intercept' 이렇게 나오면 (0,1,0) 가져다 쓸거임 #####
##### ARIMA(0,1,0) 여기 숫자 세 개만 잘 보면 됨 !!!! #####

 ARIMA(0,0,0)(0,0,0)[1] intercept   : AIC=1283.989, Time=0.08 sec
 ARIMA(0,0,1)(0,0,0)[1] intercept   : AIC=1272.821, Time=0.04 sec
 ARIMA(0,0,2)(0,0,0)[1] intercept   : AIC=1272.549, Time=0.05 sec
 ARIMA(0,0,3)(0,0,0)[1] intercept   : AIC=1267.794, Time=0.05 sec
 ARIMA(0,0,4)(0,0,0)[1] intercept   : AIC=1271.585, Time=0.07 sec
 ARIMA(0,0,5)(0,0,0)[1] intercept   : AIC=1272.544, Time=0.13 sec
 ARIMA(1,0,0)(0,0,0)[1] intercept   : AIC=1269.268, Time=0.02 sec
 ARIMA(1,0,1)(0,0,0)[1] intercept   : AIC=1271.171, Time=0.05 sec
 ARIMA(1,0,2)(0,0,0)[1] intercept   : AIC=1272.894, Time=0.05 sec
 ARIMA(1,0,3)(0,0,0)[1] intercept   : AIC=1269.559, Time=0.11 sec
 ARIMA(1,0,4)(0,0,0)[1] intercept   : AIC=1270.148, Time=0.24 sec
 ARIMA(2,0,0)(0,0,0)[1] intercept   : AIC=1271.123, Time=0.06 sec
 ARIMA(2,0,1)(0,0,0)[1] intercept   : AIC=1272.433, Time=0.12 sec
 ARIMA(2,0,2)(0,0,0)[1] intercept   : AIC=1271.290, Time=0.32 sec
 ARIMA(2,0,3)(0,0,0)[1] intercept   : AIC=1271.552, Time=0.19 sec
 ARIMA(3,0

ARIMA(order=(4, 0, 0), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [18]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.korean.values, order=(4,0,0))     ##### 여기 order=(0,1,0)에 위에서 나온 숫자 넣는거 #####

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                   42
Model:                     ARMA(4, 0)   Log Likelihood                -627.761
Method:                       css-mle   S.D. of innovations         739755.751
Date:                Sat, 11 Sep 2021   AIC                           1267.522
Time:                        01:40:47   BIC                           1277.948
Sample:                             0   HQIC                          1271.343
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       6.455e+06   1.61e+05     40.140      0.000    6.14e+06    6.77e+06
ar.L1.y        0.5341      0.139      3.837      0.000       0.261       0.807
ar.L2.y        0.0636      0.169      0.377      0.7

# 예측하기

In [19]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([7144596.61231312, 6598143.09171414]), array([739755.75078211, 838644.39615197]), array([[5694701.9834238 , 8594491.24120243],
       [4954430.27941993, 8241855.90400835]]))
